In [9]:
import win32gui
import win32con
import win32api

In [39]:
def EnumWindows():
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumWindows(_MyCallback, windows)
    return windows

In [16]:
def EnumChildWindows(hwnd):
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumChildWindows(hwnd,_MyCallback, windows)
    return windows

In [21]:
childs = EnumChildWindows('1444364')
childs

{395806: {'hex': '0x60a1e',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 854514: {'hex': '0xd09f2',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 526886: {'hex': '0x80a26',
  'ClassName': 'ApplicationFrameInputSinkWindow',
  'WindowText': ''}}

In [30]:
child_text = win32gui.GetWindowText('1444364')
print(child_text)

设置


In [50]:
# 搜索子窗口
# 枚举子窗口
hwndChildList = []
win32gui.EnumChildWindows('1444364', lambda hwnd, param: param.append(hwnd),  hwndChildList)
hwndChildList

[395806, 854514, 526886]

In [87]:
import cv2
import numpy as np
from ctypes import windll
import win32gui
import win32ui
 
 
def capture_win_alt(hwnd=None, program_windowTitle=None, program_className=None,img_name='temp.png'):
    # Adapted from https://stackoverflow.com/questions/19695214/screenshot-of-inactive-window-printwindow-win32gui
    if hwnd is None:
        windll.user32.SetProcessDPIAware()
        windows = EnumWindows()
        for key,item in windows.items() :
            if item['ClassName'] == program_className:
                if program_windowTitle != None:
                    if program_windowTitle in item['WindowText'] or program_windowTitle == item['WindowText']:
                        program_windowTitle = item['WindowText']
                        hwnd = key
                        print(key,item)
                        break;
                    else:
                        hwnd = key
                        print(key,item)
                        break;
    if not hwnd:
        hwnd = win32gui.FindWindow(program_className, program_windowTitle)
        # hwnd = win32gui.FindWindowEx(0, 0, program_className, program_windowTitle)
    if not hwnd:
        return None
    left, top, right, bottom = win32gui.GetClientRect(hwnd)
    w = right - left
    h = bottom - top
    hwnd_dc = win32gui.GetWindowDC(hwnd)
    mfc_dc = win32ui.CreateDCFromHandle(hwnd_dc)
    save_dc = mfc_dc.CreateCompatibleDC()
    bitmap = win32ui.CreateBitmap()
    bitmap.CreateCompatibleBitmap(mfc_dc, w, h)
    save_dc.SelectObject(bitmap)
    # If Special K is running, this number is 3. If not, 1
    result = windll.user32.PrintWindow(hwnd, save_dc.GetSafeHdc(), 3)
    bmpinfo = bitmap.GetInfo()
    bmpstr = bitmap.GetBitmapBits(True)
    img = np.frombuffer(bmpstr, dtype=np.uint8).reshape((bmpinfo["bmHeight"], bmpinfo["bmWidth"], 4))
    img = np.ascontiguousarray(img)[..., :-1]  # make image C_CONTIGUOUS and drop alpha channel
    if not result:  # result should be 1
        win32gui.DeleteObject(bitmap.GetHandle())
        save_dc.DeleteDC()
        mfc_dc.DeleteDC()
        win32gui.ReleaseDC(hwnd, hwnd_dc)
        raise RuntimeError(f"Unable to acquire screenshot! Result: {result}")
    cv2.imwrite(img_name, img)
    return img,w,h,hwnd

In [94]:
from cnocr import CnOcr


img,w,h,hwnd = capture_win_alt(program_className='ApplicationFrameWindow',program_windowTitle='')

ocr = CnOcr() 
result = ocr.ocr(img)

# 具体参数设置参考：https://cnocr.readthedocs.io/zh/latest/usage/


723436 {'hex': '0xb09ec', 'ClassName': 'ApplicationFrameWindow', 'WindowText': '设置'}


In [1]:
import base64
import io

from PIL import Image 

img_path = r"C:\Users\Administrator\Documents\GitHub\OmniParser\imgs\windows_vm.png"
with open(img_path,"rb") as f:
    #读取图片的二进制数据
    bin_contents = f.read()
    data = base64.b64encode(bin_contents, altchars=None)

In [2]:
data

b'...'

In [3]:
daa = {
            "base64_image": data.decode('UTF-8')
        }
import requests
response = requests.post('http://localhost:8000/parse/', json=daa)

In [5]:
response.json()

{'som_image_base64': '...',
 'parsed_content_list': [{'type': 'text',
   'bbox': [0.8589511513710022,
    0.8836081624031067,
    0.9017480611801147,
    0.9068865180015564],
   'interactivity': False,
   'content': 'Windows',
   'source': 'box_ocr_content_ocr'},
  {'type': 'text',
   'bbox': [0.9119951725006104, 0.8836081624031067, 1.0, 0.9068865180015564],
   'interactivity': False,
   'content': 'Enterprise Evaluation',
   'source': 'box_ocr_content_ocr'},
  {'type': 'text',
   'bbox': [0.8577456474304199, 0.9020368456840515, 1.0, 0.9253152012825012],
   'interactivity': False,
   'content': 'Windows License valid for 88 days',
   'source': 'box_ocr_content_ocr'},
  {'type': 'text',
   'bbox': [0.852320671081543,
    0.9214354753494263,
    0.9969861507415771,
    0.9447138905525208],
   'interactivity': False,
   'content': 'Build 26100,ge_release 240331-143',
   'source': 'box_ocr_content_ocr'},
  {'type': 'text',
   'bbox': [0.037974681705236435,
    0.9563530683517456,
    0.085